## Breast Cancer Wisconsin (Diagnostic) Data Set

Jordan DO BARREIRO

Also can be found on UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image.

Attribute Information:

* ID number
* Diagnosis (M = malignant, B = benign)

Ten real-valued features are computed for each cell nucleus:

* radius (mean of distances from center to points on the perimeter)
* texture (standard deviation of gray-scale values)
* perimeter
* area
* smoothness (local variation in radius lengths)
* compactness (perimeter^2 / area - 1.0)
* concavity (severity of concave portions of the contour)
* concave points (number of concave portions of the contour)
* symmetry
* fractal dimension ("coastline approximation" - 1)

The mean, standard error and "worst" or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.

All feature values are recoded with four significant digits.

Missing attribute values: none

Class distribution: 357 benign, 212 malignant

# Task 

* Design a neural network to predict whether a breast tumor for a new patient is benign or malignant.
* Split the data into train and test.
* No hyperparameter tuning is required.
* Train the model using the training set.
* Test the model using the testing set.
    * Use the Fscore as the evaluation metric.
    * Calculate the confusion matrix, recall, precision, and AUC of your model.
 

Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import os
import operator
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/ING5/Deep Learning")
!ls

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive
data  figures


Reading Data




In [ ]:
df = pd.read_csv("data/data_wbc.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


Clean, split and normalize the data

In [ ]:
cleaned_df = df.copy()

cleaned_df.pop('id')
cleaned_df.pop('Unnamed: 32')

cleaned_df['diagnosis'] = cleaned_df['diagnosis'].replace(['M','B'],[1,0])

cleaned_df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
cleaned_df['diagnosis'].value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

In [ ]:
X = cleaned_df
y = cleaned_df.pop('diagnosis')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=8)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(455, 30)
(455,)
(114, 30)
(114,)


Scaling numerical variables

In [ ]:
num_cols = ['radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']

SS = StandardScaler()
scaled_cols = num_cols 

X_train[scaled_cols] = SS.fit_transform(X_train[scaled_cols])
X_test[scaled_cols] = SS.transform(X_test[scaled_cols])

Define, compile and train the Neural Network

In [ ]:
model = tf.keras.Sequential([keras.layers.Dense(activation='sigmoid', units=1, input_shape=[30])])

model.compile(optimizer='sgd', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=500)

Epoch 1/500
15/15 [==============================] - 0s 1ms/step - loss: 0.4376
Epoch 2/500
15/15 [==============================] - 0s 820us/step - loss: 0.3762
Epoch 3/500
15/15 [==============================] - 0s 908us/step - loss: 0.3145
Epoch 4/500
15/15 [==============================] - 0s 1ms/step - loss: 0.2620
Epoch 5/500
15/15 [==============================] - 0s 1ms/step - loss: 0.2212
Epoch 6/500
15/15 [==============================] - 0s 1ms/step - loss: 0.1904
Epoch 7/500
15/15 [==============================] - 0s 1ms/step - loss: 0.1666
Epoch 8/500
15/15 [==============================] - 0s 966us/step - loss: 0.1485
Epoch 9/500
15/15 [==============================] - 0s 1ms/step - loss: 0.1339
Epoch 10/500
15/15 [==============================] - 0s 1ms/step - loss: 0.1224
Epoch 11/500
15/15 [==============================] - 0s 1ms/step - loss: 0.1127
Epoch 12/500
15/15 [==============================] - 0s 1ms/step - loss: 0.1045
Epoch 13/500
15/15 [===========

In [ ]:
pred = np.round(model.predict(X_test)).astype(int)

print(pred[:15])
print(y_test[:15])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]]
325    0
557    0
475    0
308    0
553    0
159    0
290    0
146    1
431    0
527    0
442    0
309    0
384    0
328    1
56     1
Name: diagnosis, dtype: int64


F1 Score

In [ ]:
print(f1_score(y_test, pred, average=None))

[0.98529412 0.97826087]


Confusion matrix : 

True Positive False Positive

False Negative  True Negative

In [ ]:
confusion_matrix(y_test, pred)

array([[67,  1],
       [ 1, 45]])

Recall

In [ ]:
recall_score(y_test, pred, average=None)

array([0.98529412, 0.97826087])

Precision

In [ ]:
precision_score(y_test, pred, average=None)

array([0.98529412, 0.97826087])

AUC

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, pred)
auc(fpr, tpr)

0.9817774936061381

Classification report

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99        68
           1       0.98      0.98      0.98        46

    accuracy                           0.98       114
   macro avg       0.98      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114

